# Instalación de librerías

In [4]:
#Descargamos e instalamos Flask
!pip install flask-ngrok

# Librerías

In [5]:
#Librerías Base
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [6]:
#Importamos la librería para leer modelos
from keras.models import load_model

In [7]:
#Importamos las librerías para la aplicacion web
import flask
from flask import Flask
from flask_ngrok import run_with_ngrok
from flask import request
from flask import render_template

# Conexión al repositorio de datos

In [8]:
#Accedemos a Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Lectura del modelo

In [9]:
#Leemos el modelo
modelLeido = load_model('/content/drive/MyDrive/Data/modelo_red_neuronal.h5')

In [10]:
#Leemos el archivo del escalador
metadataLeida = np.load('/content/drive/MyDrive/Data/metadata_red_neuronal.npz', allow_pickle=True)

In [11]:
#Extraemos el escalador
scLeido = metadataLeida['arr_0'][()]['scaler']
scLeido

MinMaxScaler(copy=True, feature_range=(0, 1))

In [12]:
#Extraemos la metadata
categoriasLeida = metadataLeida['arr_0'][()]['categories']
categoriasLeida

['setosa', 'versicolor', 'virginica']

# Uso del modelo

In [13]:
#Implementamos la funcion que utiliza el modelo
def usar_modelo(sepal_length, sepal_width, petal_length, petal_width):
  #Convertimos a números los parámetros recibidos desde la web
  f_sepal_length = float(sepal_length)
  f_sepal_width = float(sepal_width)
  f_petal_length = float(petal_length)
  f_petal_width = float(petal_width)

  #Definimos los features para la predicción en un array
  x = [f_sepal_length, f_sepal_width, f_petal_length, f_petal_width]
  
  #Aplicamos el modelo para obtener la predicción
  #Colocamos los features como un array, ya que podemos enviarle más de un registro a la vez
  y = modelLeido.predict([x])

  #Como es un modelo multi-clase, extraemos la posición máxima
  indice_max = y.argmax()

  #De la metada leída, verificamos a qué cateogoría corresponde
  categoria_prediccion = categoriasLeida[indice_max]

  #Devolvemos la respuesta
  return categoria_prediccion

# Creación de la aplicación web

In [14]:
#Creamos una aplicación web con Flask
#"__name__" es el nombre de nuestro proyecto de Google Colab
#"template_folder" es la ruta en donde estarán desplegados nuestros formularios
#Notar que en el caso de Google Colab + Google Drive, la ruta empieza desde "drive" sin "/"
app = Flask(__name__, template_folder='drive/MyDrive/Data')

In [15]:
#Indicamos que queremos ejecutar nuestra aplicación web con NGrok para poder ver la web
run_with_ngrok(app)

In [16]:
#Creamos la función que define el contenido de la raíz de la web (/)
#De manera estándar a la raíz se le conoce como "home"
@app.route("/")
def home():
  return render_template("formulario_modelo_red_neuronal.html")

In [17]:
#Creamos otra web en el servidor
@app.route("/resultado", methods=['GET', 'POST'])
def resultado():
  #Capturamos los parametros de la web
  sepal_length = request.form['sepal_length']
  sepal_width = request.form['sepal_width']
  petal_length = request.form['petal_length']
  petal_width = request.form['petal_width']

  #Colocamos los parametros en una variable
  parametros = "["+sepal_length+", "+sepal_width+", "+petal_length+", "+petal_width+"]"

  #Enviamos los parametros al modelo
  resultado = str(usar_modelo(sepal_length, sepal_width, petal_length, petal_width))

  #Mostramos la web con los parametros y el resultado
  return render_template("resultado.html", parametros=parametros, resultado=resultado)

# Ejecucion de la aplicación web

In [ ]:
#Ejecutamos la aplicación
#Para abrir la web deberemos dar clic en el enlace que aparezca
#Para detener el servidor deberemos detener la ejecución de esta sección del código
#Algunos valores de prueba: [1.101218, 1.052339, 1.076814, 1.098520, 1.102388, 1.137163, 1.135161, 1.055935, 0.943724, 0.758063, 0.782767]
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4455-35-243-223-227.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [29/Oct/2021 02:00:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 02:00:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 02:01:00] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2021 02:01:13] "POST /resultado HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 02:01:14] "GET /resultado HTTP/1.1" 400 -
127.0.0.1 - - [29/Oct/2021 02:01:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 02:01:39] "POST /resultado HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 02:01:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 02:01:53] "POST /resultado HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 02:01:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 02:02:03] "POST /resultado HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 02:02:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 02:02:53] "POST /resultado HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 02:04:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 02:05:05] "POST /resultado HTTP/1.1" 200 -
127.0.0.1 - - [29/Oc